<a href="https://colab.research.google.com/github/adile04/machine-learning-models/blob/main/NLP_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sentiment Analysis

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

raw_train_set,raw_valid_set,raw_test_set = tfds.load(
    name = 'imdb_reviews',
    split = ["train[:90%]","train[90%:]","test"],
    as_supervised=True
)
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000,seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.IO5D6T_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.IO5D6T_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.IO5D6T_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
for review, label in raw_train_set.take(4):
  print(review.numpy().decode("utf-8")[:200],"...")
  print("Label:",label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0
Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun ...
Label: 0
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf ...
Label: 1


In [ ]:
vocab_size = 1000

text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size,
                                                   standardize="lower_and_strip_punctuation")
text_vec_layer.adapt(train_set.map(lambda review,label:review))


In [ ]:
text_vec_layer.get_vocabulary()[:50]

['',
 '[UNK]',
 np.str_('the'),
 np.str_('and'),
 np.str_('a'),
 np.str_('of'),
 np.str_('to'),
 np.str_('is'),
 np.str_('in'),
 np.str_('it'),
 np.str_('i'),
 np.str_('this'),
 np.str_('that'),
 np.str_('br'),
 np.str_('was'),
 np.str_('as'),
 np.str_('for'),
 np.str_('with'),
 np.str_('movie'),
 np.str_('but'),
 np.str_('film'),
 np.str_('on'),
 np.str_('not'),
 np.str_('you'),
 np.str_('are'),
 np.str_('his'),
 np.str_('have'),
 np.str_('he'),
 np.str_('be'),
 np.str_('one'),
 np.str_('its'),
 np.str_('at'),
 np.str_('all'),
 np.str_('by'),
 np.str_('an'),
 np.str_('they'),
 np.str_('from'),
 np.str_('who'),
 np.str_('so'),
 np.str_('like'),
 np.str_('her'),
 np.str_('or'),
 np.str_('just'),
 np.str_('about'),
 np.str_('has'),
 np.str_('if'),
 np.str_('out'),
 np.str_('some'),
 np.str_('there'),
 np.str_('what')]

In [ ]:
list(map(str,text_vec_layer.get_vocabulary()[:50]))

['',
 '[UNK]',
 'the',
 'and',
 'a',
 'of',
 'to',
 'is',
 'in',
 'it',
 'i',
 'this',
 'that',
 'br',
 'was',
 'as',
 'for',
 'with',
 'movie',
 'but',
 'film',
 'on',
 'not',
 'you',
 'are',
 'his',
 'have',
 'he',
 'be',
 'one',
 'its',
 'at',
 'all',
 'by',
 'an',
 'they',
 'from',
 'who',
 'so',
 'like',
 'her',
 'or',
 'just',
 'about',
 'has',
 'if',
 'out',
 'some',
 'there',
 'what']

In [ ]:
text_vec_layer(['it was a great movie',
                'it was'])

<tf.Tensor: shape=(2, 5), dtype=int64, numpy=
array([[ 9, 14,  4, 86, 18],
       [ 9, 14,  0,  0,  0]])>

In [ ]:
embed_layer = tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=64)
embed_layer(text_vec_layer(['it was a great movie']))

<tf.Tensor: shape=(1, 5, 64), dtype=float32, numpy=
array([[[ 1.45667084e-02,  4.11798619e-02, -2.45891940e-02,
          3.14583816e-02, -3.96800265e-02,  4.22104858e-02,
         -2.32768059e-03, -1.07531622e-03, -1.54242143e-02,
         -1.28259063e-02, -1.31056309e-02,  1.43373944e-02,
         -4.51291911e-02, -2.64880899e-02, -2.64091417e-03,
         -3.80181558e-02, -3.73166092e-02, -7.23075122e-04,
         -4.96066809e-02, -1.97515246e-02,  4.14348766e-03,
          4.40994762e-02,  3.38647850e-02,  4.89143468e-02,
         -1.54100731e-03,  2.97305472e-02,  7.41062313e-03,
          2.08112039e-02,  4.70893495e-02, -2.18798406e-02,
          4.05972116e-02,  6.14923239e-03, -4.23194766e-02,
          2.23577060e-02, -2.62035057e-03, -2.99242139e-02,
          4.60121669e-02,  1.14380009e-02,  2.61749141e-02,
          1.92694776e-02,  2.79623382e-02,  1.51238479e-02,
          2.52903365e-02, -2.27882508e-02, -2.80325413e-02,
         -9.56274569e-04,  2.34966986e-02, -2.64

In [ ]:
embed_size = 128
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(input_dim = vocab_size,output_dim=embed_size,mask_zero=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

In [ ]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer= tf.keras.optimizers.Nadam(learning_rate=1e-3),
    metrics = ['accuracy']
)
model.fit(train_set,validation_data = valid_set,epochs=2)

Epoch 1/2
704/704 ━━━━━━━━━━━━━━━━━━━━ 26s 32ms/step - accuracy: 0.6044 - loss: 0.6573 - val_accuracy: 0.8220 - val_loss: 0.3848
Epoch 2/2
704/704 ━━━━━━━━━━━━━━━━━━━━ 47s 42ms/step - accuracy: 0.8523 - loss: 0.3407 - val_accuracy: 0.8652 - val_loss: 0.3082


In [ ]:
embedding_weights = model.layers[1].get_weights()[0]

In [ ]:
import numpy as np
np.savetxt("embeddings.tsv",embedding_weights,delimiter="\t")

In [ ]:
vocab = text_vec_layer.get_vocabulary()

with open("metadata.tsv","w",encoding = "utf-8") as f:
  for word in vocab:
    word = word if word.strip()!="" else "<PAD>"
    f.write(f"{word}\n")

#Generating Shakespearean Text Using a Character RNN

In [ ]:
import tensorflow as tf
shakespeare_url = "https://homl.info/shakespeare"

filepath = tf.keras.utils.get_file("shakespeare.txt",shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [ ]:
''.join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [ ]:
text_vec_layer  = tf.keras.layers.TextVectorization(split='character',
                                                    standardize = 'lower')
text_vec_layer.adapt(shakespeare_text)
encoded = text_vec_layer([shakespeare_text][0])

In [ ]:
encoded -=2
n_tokens = text_vec_layer.vocabulary_size()-2
dataset_size = len(encoded)

In [ ]:
n_tokens

39

In [ ]:
dataset_size

1115394

In [ ]:
#input -> Before we proceed any furthe
#output -> efore we proceed any further

In [ ]:
def to_dataset(sequence , length ,seed =None,shuffle = False,batch_size = 32):
  ds = tf.data.Dataset.from_tensor_slices(sequence)
  ds = ds.window(length+1,shift=1,drop_remainder = True)
  ds = ds.flat_map(lambda window_ds: window_ds.batch(length+1))
  if shuffle :
    ds = ds.shuffle(100_000,seed=seed)
  ds = ds.batch(batch_size)
  return ds.map(lambda window:(window[:,:-1],window[:,1:]))

In [ ]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000],length=length,shuffle=True,
                       seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000],length=length)
test_set = to_dataset(encoded[1_060_000:],length=length)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = vocab_size,output_dim=16),
    tf.keras.layers.GRU(128,return_sequences = True),
    tf.keras.layers.Dense(vocab_size,activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",
              metrics = ["accuracy"])

model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model.keras",
    monitor="val_accuracy",
    save_best_only = True)

history = model.fit(train_set,validation_data=valid_set,epochs=1,
                    callbacks=[model_ckpt])

   8831/Unknown 168s 16ms/step - accuracy: 0.3538 - loss: 2.2835

KeyboardInterrupt: 

In [ ]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X:X-2),
    model
])

In [ ]:
y_proba = shakespeare_model.predict(tf.constant(["To be or not to b"]))[0,-1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred+2]

#Generating Fake Shakespeare text

In [ ]:
log_probas = tf.math.log([[0.5,0.3,0.2]])
tf.random.categorical(log_probas,num_samples=8)

In [ ]:
def next_char(text,temperature):
  text = tf.constant([text])
  y_proba = shakespeare_model.predict(text)[0,-1:]
  rescaled_logits = tf.math.log(y_proba)/temperature
  char_id = tf.random.categorical(rescaled_logits,num_samples=1)[0,0]
  return text_vec_layer.get_vocabulary()[char_id+2]

In [ ]:
def extent_text(text,chars=50,temperature = 1):
  for _ in range(chars):
    text +=next_char(text,temperature)
  return text

In [ ]:
extent_text('to be or not to be',chars = 100,temperature = 1)

#An Encoder-Decoder Network for Neural Machine Translation

In [ ]:
from pathlib import Path
import tensorflow as tf

url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets", extract=True)

# Final corrected path
spa_txt_path = Path(path).parent / "spa-eng_extracted" / "spa-eng" / "spa.txt"

# Read the file
text = spa_txt_path.read_text(encoding='utf-8')
print(text[:500])  # Print first 500 characters as a quick check

2638744/2638744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Go.	Ve.
Go.	Vete.
Go.	Vaya.
Go.	Váyase.
Hi.	Hola.
Run!	¡Corre!
Run.	Corred.
Who?	¿Quién?
Fire!	¡Fuego!
Fire!	¡Incendio!
Fire!	¡Disparad!
Help!	¡Ayuda!
Help!	¡Socorro! ¡Auxilio!
Help!	¡Auxilio!
Jump!	¡Salta!
Jump.	Salte.
Stop!	¡Parad!
Stop!	¡Para!
Stop!	¡Pare!
Wait!	¡Espera!
Wait.	Esperen.
Go on.	Continúa.
Go on.	Continúe.
Hello!	Hola.
I ran.	Corrí.
I ran.	Corría.
I try.	Lo intento.
I won!	¡He ganado!
Oh no!	¡Oh, no!
Relax.	Tomátelo con soda.
Smile.	Sonríe.
Attack!	¡Al ataque!
Attack!	¡Atacad!
Ge


In [ ]:
import numpy as np

text = text.replace("¡","").replace("¿","")
pairs = [line.split('\t')for line in text.splitlines()]
np.random.seed(42)
np.random.shuffle(pairs)
sentence_en ,sentences_es = zip(*pairs)

In [ ]:
pairs

[['How boring!', 'Qué aburrimiento!'],
 ['I love sports.', 'Adoro el deporte.'],
 ['Would you like to swap jobs?',
  'Te gustaría que intercambiemos los trabajos?'],
 ['My mother did nothing but weep.', 'Mi madre no hizo nada sino llorar.'],
 ['Croatia is in the southeastern part of Europe.',
  'Croacia está en el sudeste de Europa.'],
 ['I have never eaten a mango before.', 'Nunca he comido un mango.'],
 ['Tell the taxi driver to drive faster.',
  'Decile al taxista que maneje más rápido.'],
 ['Tom and I work together.', 'Tom y yo trabajamos juntos.'],
 ['I would prefer an honorable death.', 'Preferiría una muerte honorable.'],
 ['Tom married a much younger woman.',
  'Tom se ha casado con una mujer mucho más joven.'],
 ["It couldn't happen to me.", 'A mí no podría pasarme.'],
 ["Tom isn't going to marry you.", 'Tom no se va a casar contigo.'],
 ["Can you believe it? He's even lazier than me.",
  'Puedes creerlo? Él es aún más flojo que yo.'],
 ['She has hundreds of books.', 'Ella tie

In [ ]:
for i in range(3):
  print(sentence_en[i],"=>",sentences_es[i])

How boring! => Qué aburrimiento!
I love sports. => Adoro el deporte.
Would you like to swap jobs? => Te gustaría que intercambiemos los trabajos?


In [ ]:
vocab_size = 1000
max_lenght = 50

# Extract sentences_en and sentences_es from pairs
sentence_en, sentences_es = zip(*pairs)  # This line unpacks your 'pairs' into English and Spanish sentences

text_vec_layer_en = tf.keras.layers.TextVectorization(
    vocab_size,output_sequence_length = max_lenght
)
text_vec_layer_en.adapt(sentence_en)

text_vec_layer_es = tf.keras.layers.TextVectorization(
    vocab_size,output_sequence_length = max_lenght
)
text_vec_layer_es.adapt([f'startofseq {s} endofseq' for s in sentences_es])

In [ ]:
text_vec_layer_en.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('the'),
 np.str_('i'),
 np.str_('to'),
 np.str_('you'),
 np.str_('tom'),
 np.str_('a'),
 np.str_('is'),
 np.str_('he')]

In [ ]:
text_vec_layer_es.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('startofseq'),
 np.str_('endofseq'),
 np.str_('de'),
 np.str_('que'),
 np.str_('a'),
 np.str_('no'),
 np.str_('tom'),
 np.str_('la')]

In [ ]:
X_train = tf.constant(sentence_en[:100_000])
X_valid = tf.constant(sentence_en[100_000:])
X_train_dec = tf.constant([f'startofseq {s}'for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f'startofseq {s}'for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f'{s} endofseq'for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f'{s} endofseq'for s in sentences_es[100_000:]])

In [ ]:
tf.random.set_seed(42)
encoder_inputs = tf.keras.layers.Input(shape=[],dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[],dtype=tf.string)

In [ ]:
embed_size=128

encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)

encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,output_dim=embed_size,mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,output_dim=embed_size,mask_zero=True)

encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

In [ ]:
encoder=tf.keras.layers.LSTM(512,return_state=True)
encoder_outputs,*encoder_state = encoder(encoder_embeddings)

In [ ]:
decoder = tf.keras.layers.LSTM(512,return_sequences=True)
decoder_outputs = decoder(decoder_embeddings,initial_state = encoder_state)

In [ ]:
output_layer  = tf.keras.layers.Dense(vocab_size,activation = 'softmax')
Y_proba = output_layer(decoder_outputs)

In [ ]:
model = tf.keras.Model(inputs = [encoder_inputs,decoder_inputs],
                       outputs = [Y_proba])

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'nadam',
              metrics = ['accuracy'])

model.fit((X_train,X_train_dec),Y_train,epochs = 3,
          validation_data = ((X_valid,X_valid_dec),Y_valid))

Epoch 1/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 85s 25ms/step - accuracy: 0.0519 - loss: 3.4823 - val_accuracy: 0.0756 - val_loss: 2.0967
Epoch 2/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 74s 24ms/step - accuracy: 0.0799 - loss: 1.9269 - val_accuracy: 0.0879 - val_loss: 1.5938
Epoch 3/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 91s 27ms/step - accuracy: 0.0918 - loss: 1.4512 - val_accuracy: 0.0929 - val_loss: 1.4082


In [ ]:
def translate(sentence_en):
    translation=''
    max_length=50
    for word_idx in range(max_length):
        X=tf.constant([sentence_en])
        X_dec=tf.constant(['startsofseq'+ translation])
        y_proba=model.predict((X,X_dec))[0,word_idx]
        predicted_word_id=np.argmax(y_proba)
        predicted_word=text_vec_layer_es.get_vocabulary()[predicted_word_id]
        if predicted_word=='endofseq':
            break
        translation+=' '+predicted_word
    return translation.strip()


In [ ]:
translate('i like soccer')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


'me gusta el fútbol'

# Bidirectional Encoder

In [ ]:
tf.random.set_seed(42)
encoder=tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256,return_state=True)
)

In [ ]:
class ConcatenateStates(tf.keras.layers.Layer):
  def __init__(self):
      super().__init__()
  def call(self,encoder_state):
      return [tf.concat(encoder_state[::2],axis=-1),
              tf.concat(encoder_state[1::2],axis=-1)]
encoder_outputs,*encoder_state=encoder(encoder_embeddings)
concat_states=ConcatenateStates()
encoder_state=concat_states(encoder_state)

In [ ]:
decoder=tf.keras.layers.LSTM(512,return_sequences=True)
decoder_outputs=decoder(decoder_embeddings,initial_state=encoder_state)

output_layer=tf.keras.layers.Dense(vocab_size,activation='softmax')
Y_proba=output_layer(decoder_outputs)
model=tf.keras.Model(inputs=[encoder_inputs,decoder_inputs],
                     outputs=[Y_proba])
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])
model.fit((X_train,X_train_dec),Y_train,epochs=3,
          validation_data=((X_valid,X_valid_dec),Y_valid))

Epoch 1/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 87s 26ms/step - accuracy: 0.0652 - loss: 2.8138 - val_accuracy: 0.0897 - val_loss: 1.5336
Epoch 2/3


KeyboardInterrupt: 

In [ ]:
translate('i like soccer and also going to the beach')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


'me gusta el béisbol y no [UNK] a la playa'

beam searhc

In [ ]:
def beam_search(sentence_en, beam_width, verbose=False,max_length=50):
    X = tf.constant([sentence_en])  # encoder input
    X_dec = tf.constant(["startofseq"])  # decoder input
    y_proba = model.predict((X, X_dec))[0, 0]  # first token's probas
    top_k = tf.math.top_k(y_proba, k=beam_width)
    top_translations = [  # list of best (log_proba, translation)
        (np.log(word_proba), text_vec_layer_es.get_vocabulary()[word_id])
        for word_proba, word_id in zip(top_k.values, top_k.indices)
    ]

    # extra code – displays the top first words in verbose mode
    if verbose:
        print("Top first words:", top_translations)

    for idx in range(1, max_length):
        candidates = []
        for log_proba, translation in top_translations:
            if translation.endswith("endofseq"):
                candidates.append((log_proba, translation))
                continue  # translation is finished, so don't try to extend it
            X = tf.constant([sentence_en])  # encoder input
            X_dec = tf.constant(["startofseq " + translation])  # decoder input
            y_proba = model.predict((X, X_dec))[0, idx]  # last token's proba
            for word_id, word_proba in enumerate(y_proba):
                word = text_vec_layer_es.get_vocabulary()[word_id]
                candidates.append((log_proba + np.log(word_proba),
                                   f"{translation} {word}"))
        top_translations = sorted(candidates, reverse=True)[:beam_width]

        # extra code – displays the top translation so far in verbose mode
        if verbose:
            print("Top translations so far:", top_translations)

        if all([tr.endswith("endofseq") for _, tr in top_translations]):
            return top_translations[0][1].replace("endofseq", "").strip()

In [ ]:
sentence_en='I love cats and dogs'
translate(sentence_en)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


'me gustan los perros y los gatos'

In [ ]:
beam_search(sentence_en,beam_width=3,verbose=True)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Top first words: [(np.float32(-0.5444548), np.str_('me')), (np.float32(-2.5361273), '[UNK]'), (np.float32(-2.8155067), np.str_('la'))]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Top translations so far: [(np.float32(-1.7659538), 'me gustan'), (np.float32(-1.9788779), 'me [UNK]'), (np.float32(-2.1023843), 'me encanta')]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Top translations so far: [(np.float32(-1.9992012), 'me gustan los'), (np.float32(-3.189676), 'me [UNK] a'), (np.float32(-3.2396078), 'me [UNK] los')]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Top translations so far: [(np.float32(-2.8495817), 'me gustan los perros'), (np.float32(-3.1435356), 'me gustan los gatos'), (np.float32(-3.492002), 'me [UNK] a los')]
1/1 ━━━━━━━━━━━━━━━━━━━━

'me gustan los gatos y los gatos'

# Attention Mechanisms

In [ ]:
tf.random.set_seed(42)
encoder_inputs=tf.keras.layers.Input(shape=[],dtype=tf.string)
decoder_inputs=tf.keras.layers.Input(shape=[],dtype=tf.string)

In [ ]:
embed_size=128
encoder_input_ids=text_vec_layer_en(encoder_inputs)
decoder_input_ids=text_vec_layer_es(decoder_inputs)

encoder_embedding_layer=tf.keras.layers.Embedding(vocab_size,
                                                  output_dim=embed_size)
decoder_embedding_layer=tf.keras.layers.Embedding(vocab_size,
                                                  output_dim=embed_size
                                                  )
encoder_embeddings=encoder_embedding_layer(encoder_input_ids)
decoder_embeddings=decoder_embedding_layer(decoder_input_ids)

In [ ]:
encoder=tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256,return_sequences=True,return_state=True)
)
encoder_outputs,*encoder_state=encoder(encoder_embeddings)
encoder_state=concat_states(encoder_state)

In [ ]:
decoder=tf.keras.layers.LSTM(512,return_sequences=True)
decoder_outputs=decoder(decoder_embeddings,initial_state=encoder_state)

In [ ]:
attention_layer=tf.keras.layers.Attention()
attention_outputs=attention_layer([decoder_outputs,encoder_outputs])
output_layer=tf.keras.layers.Dense(vocab_size,activation='softmax')
Y_proba=output_layer(attention_outputs)

In [ ]:
model=tf.keras.Model(inputs=[encoder_inputs,decoder_inputs],
                     outputs=[Y_proba])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])

model.fit((X_train,X_train_dec),Y_train,epochs=3,
          validation_data=((X_valid,X_valid_dec),Y_valid))

Epoch 1/3
 121/3125 ━━━━━━━━━━━━━━━━━━━━ 1:16 25ms/step - accuracy: 0.8256 - loss: 1.7728

KeyboardInterrupt: 

# Transfer model

In [ ]:
import tensorflow as tf

vocab_size = 15000
max_length = 50
embed_size = 128
num_heads = 5
ff_dim = 512

# input layers
encoder_inputs = tf.keras.Input(shape=(None,), dtype=tf.int32, name='encoder_inputs')
decoder_inputs = tf.keras.Input(shape=(None,), dtype=tf.int32, name='decoder_inputs')

# embedding layer
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                                  output_dim=embed_size,
                                                  mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                                  output_dim=embed_size,
                                                  mask_zero=True)

encoder_embeddings = encoder_embedding_layer(encoder_inputs)
decoder_embeddings = decoder_embedding_layer(decoder_inputs)

# positional embedding
pos_embedding_layer = tf.keras.layers.Embedding(max_length, embed_size)

# Generate position indices using Lambda layers and then call pos_embedding_layer
positions_encoder = tf.keras.layers.Lambda(lambda x: tf.range(start=0, limit=tf.shape(x)[1], delta=1))(encoder_inputs)  # Call the Lambda layer with encoder_inputs
positions_decoder = tf.keras.layers.Lambda(lambda x: tf.range(start=0, limit=tf.shape(x)[1], delta=1))(decoder_inputs)  # Call the Lambda layer with decoder_inputs

pos_embed_enc = pos_embedding_layer(positions_encoder)  # Pass the output tensor to pos_embedding_layer
pos_embed_dec = pos_embedding_layer(positions_decoder)

#add tokens and postional embeddings
encoder_embed=encoder_embeddings+pos_embed_enc
decoder_embed=decoder_embeddings+pos_embed_dec

#encoder_self attention
encoder_attention=tf.keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                     key_dim=embed_size)(encoder_embed,encoder_embed)
encoder_attention=tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoder_embed+encoder_attention)

#encoder feed forward
encoder_ff=tf.keras.layers.Dense(ff_dim,activation='relu')(encoder_attention)
encoder_ff=tf.keras.layers.Dense(embed_size)(encoder_ff)
encoder_outputs=tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoder_attention+encoder_ff)

#decoder self attention
casual_mask=tf.keras.layers.Lambda(
    lambda x:tf.linalg.band_part(tf.ones((tf.shape(x)[1],tf.shape(x)[1])),-1,0))(decoder_inputs)
decoder_attention=tf.keras.layers.MultiHeadAttention(num_heads=num_heads,key_dim=embed_size)(decoder_embed,decoder_embed,attention_mask=casual_mask)
decoder_attention=tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_embed+decoder_attention)

#decoder encoder cross attention
cross_attention=tf.keras.layers.MultiHeadAttention(num_heads=num_heads,key_dim=embed_size)(decoder_attention,
                                                                                           encoder_outputs,encoder_outputs)
decoder_cross=tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_attention+cross_attention)

#decoder feed-forward
decoder_ff=tf.keras.layers.Dense(ff_dim,activation='relu')(decoder_cross)
decoder_ff=tf.keras.layers.Dense(embed_size)(decoder_ff)
decoder_outputs=tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_cross+decoder_ff)

#final output layer
output_logits=tf.keras.layers.Dense(vocab_size,activation='softmax')(decoder_outputs)

transformer=tf.keras.Model([encoder_inputs,decoder_inputs],output_logits)

In [ ]:
transformer.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='nadam',
    metrics=['accuracy']
)

In [ ]:
vocab_size=15000
max_length=50
text_vec_layer_en=tf.keras.layers.TextVectorization(
    vocab_size,output_sequence_length=max_length,
    pad_to_max_tokens=True
)
text_vec_layer_es=tf.keras.layers.TextVectorization(
    vocab_size,output_sequence_length=max_length,
    pad_to_max_tokens=True
)
text_vec_layer_en.adapt(sentence_en)
text_vec_layer_es.adapt([f'startofseq {s} endofseq' for s in sentences_es])

In [ ]:
X_train_padded=tf.keras.preprocessing.sequence.pad_sequences(
    text_vec_layer_en(X_train).numpy(),padding='post',maxlen=max_length
)
X_train_dec_padded=tf.keras.preprocessing.sequence.pad_sequences(
    text_vec_layer_en(X_train_dec).numpy(),padding='post',maxlen=max_length
)
X_valid_padded=tf.keras.preprocessing.sequence.pad_sequences(
    text_vec_layer_en(X_valid).numpy(),padding='post',maxlen=max_length
)
X_valid_dec_padded=tf.keras.preprocessing.sequence.pad_sequences(
    text_vec_layer_en(X_valid_dec).numpy(),padding='post',maxlen=max_length
)
X_train_padded=tf.constant(X_train_padded)
X_train_dec_padded=tf.constant(X_train_dec_padded)
X_valid_padded=tf.constant(X_valid_padded)
X_valid_dec_padded=tf.constant(X_valid_dec_padded)
transformer.fit(
    (X_train_padded,X_train_dec_padded),
    Y_train,
    epochs=3,
    validation_data=((X_valid_padded,X_valid_dec_padded),Y_valid)
)

Epoch 1/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 97s 26ms/step - accuracy: 0.9042 - loss: 0.8045 - val_accuracy: 0.9348 - val_loss: 0.2706
Epoch 2/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 131s 25ms/step - accuracy: 0.9370 - loss: 0.2570 - val_accuracy: 0.9384 - val_loss: 0.2448
Epoch 3/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 82s 25ms/step - accuracy: 0.9428 - loss: 0.2211 - val_accuracy: 0.9402 - val_loss: 0.2373


In [ ]:
import numpy as np

def translate(sentence_en):
    # Tokenize and pad encoder input
    X = text_vec_layer_en(tf.constant([sentence_en]))
    X = tf.keras.preprocessing.sequence.pad_sequences(X.numpy(), padding="post", maxlen=max_length)

    # Start token
    start_token = text_vec_layer_es(['startofseq'])[0][0]
    end_token = text_vec_layer_es(['endofseq'])[0][0]

    # Decoder input initialized with just the start token
    decoder_input = [start_token]

    for _ in range(max_length):
        decoder_input_padded = tf.keras.preprocessing.sequence.pad_sequences(
            [decoder_input], maxlen=max_length, padding="post"
        )

        y_proba = transformer.predict((X, decoder_input_padded), verbose=0)[0, len(decoder_input)-1]
        predicted_word_id = np.argmax(y_proba)

        if predicted_word_id == end_token:
            break

        decoder_input.append(predicted_word_id)

    # Map tokens back to words
    vocab = text_vec_layer_es.get_vocabulary()
    translated_words = [vocab[token] for token in decoder_input[1:]]  # skip start token

    return ' '.join(translated_words)